In [1]:
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

In [2]:
from calculate_model_statistics import calculate_RMSE_MSE

def create_features(in_data):
    in_data['DATE_TIME'] = pd.to_datetime(in_data['DATE_TIME'])
    in_data["DATE"] = in_data['DATE_TIME'].dt.date
    in_data["HOUR"] = in_data['DATE_TIME'].dt.hour
    in_data["MINUTE"] = in_data['DATE_TIME'].dt.minute
    in_data["DAY_OF_YEAR"] = in_data['DATE_TIME'].dt.dayofyear
    in_data["DAY_OF_WEEK"] = in_data['DATE_TIME'].dt.dayofweek
    in_data["MONTH"] = in_data['DATE_TIME'].dt.month
    in_data["DAY_OF_MONTH"] = in_data['DATE_TIME'].dt.day
predict_column = 'DC_POWER'

#PERFORMANCE VALIDATION ON TEST DATA
test_data1 = pd.read_csv('../Data/Train_Plant1_added_features.csv')
test_data2 = pd.read_csv('../Data/Test_Plant2_added_features.csv')
 
#BEST 2 plant model
#No training

#DO DATA PREP FOR TRAIN
#no prep

#TRAIN BEST MODEL AGAIN
#No model

#CREATE TEST_DATA FEATURES
create_features(test_data1)
create_features(test_data2)

#DO TEST DATA PREDICTIONS
test_predictions1 = np.mean(test_data1[['DC_POWER_PER_INVERTER_DAY_SHIFT_MINUS_1', 'DC_POWER_PER_INVERTER_DAY_SHIFT_MINUS_2', 'DC_POWER_PER_INVERTER_DAY_SHIFT_MINUS_3']], axis = 1)
test_predictions2 = np.mean(test_data2[['DC_POWER_PER_INVERTER_DAY_SHIFT_MINUS_1', 'DC_POWER_PER_INVERTER_DAY_SHIFT_MINUS_2', 'DC_POWER_PER_INVERTER_DAY_SHIFT_MINUS_3']], axis = 1)

#INDIVIDUAL PERIODS PREDICTIONS
print('\n#PLANT 1')
calculate_RMSE_MSE(test_data1, predict_column, test_predictions1)

print('\n#PLANT 2')
calculate_RMSE_MSE(test_data2, predict_column, test_predictions2)


#PLANT 1
daily predictions:
 RMSE - 4526646363222.0; MAE - 1361016.0
individual measurements:
 RMSE - 4469136.0; MAE - 940.0

#PLANT 2
daily predictions:
 RMSE - 34958148529.0; MAE - 133949.0
individual measurements:
 RMSE - 57728.0; MAE - 132.0
